In [1]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse

import os
import sys
import django
from user.models import User

from django.utils import timezone
import datetime
import json
import random
import string

os.chdir('/home/wxy/prj/Tostada')
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from tool.models import AuthorizationLevel

In [8]:
list(AuthorizationLevel.objects.values_list("title").filter(id__in=[3,6,11,12,17,18,19,20,21,22,23,24,25,26,29,31,32,33,34,35,36,37,38,39,44,45]))

[('View Channel',),
 ('Create Expressions',),
 ('Create Invite',),
 ('Change Nickname',),
 ('Send Messages',),
 ('Send Messages in Threads',),
 ('Create Public Threads',),
 ('Create Private Threads',),
 ('Embed Links',),
 ('Attach Files',),
 ('Add Reactions',),
 ('Use External Emoji',),
 ('Use External Stickers',),
 ('Mention @everyone, @here, and All Roles',),
 ('Read Message History',),
 ('Use Application Commands',),
 ('Send Voice Messages',),
 ('Connect',),
 ('Speak',),
 ('Video',),
 ('Use Activities',),
 ('Use Soundboard',),
 ('Use External Sounds',),
 ('Use Voice Activity',),
 ('Set Voice Channel Status',),
 ('Create Events',)]

In [ ]:
(User.objects.get()).__class__.__name__